## Import of the libraries

In [1]:
# Import of all the required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats
import seaborn as sns
import pickle

## Import of the Dataset

The data will be imported from a pickle file saved after the data analysis notebook

In [2]:
# Connecting the Google drive with Colab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Import of the train and test datasets into the Panda dataframe from Google drive 
# To load data from the local drive the location below can be changed accordingly


train_df = pickle.load(open("/content/drive/My Drive/Colab Notebooks/train_df_final.pkl", "rb"))
test_df = pickle.load(open("/content/drive/My Drive/Colab Notebooks/test_df_final.pkl", "rb"))

train_df
test_df

,V1,V4,V5,V7,V8,V9,V10,V13,V14,V16,V20,V21,V22,V24,V27,V28,Amount,Class
Id,,,,,,,,,,,,,,,,,,
1,-0.584007,0.001509,0.292260,1.120755,-0.166826,-0.501695,-0.694423,-0.276469,0.284019,0.265888,-0.106804,0.167209,0.361133,0.375231,0.102098,0.144502,42.81,0
2,1.326714,-0.979830,-0.625116,-0.636415,0.050208,-0.785860,0.815473,-1.025594,0.271240,1.751346,0.164916,0.109578,-0.011447,-0.859332,-0.021529,0.003230,75.00,0
3,2.097691,-0.011746,0.597501,0.130716,-0.174237,0.383371,0.171848,0.344720,0.589561,0.230686,-0.192933,0.233496,0.825427,-1.115397,-0.020643,-0.079921,2.29,0
4,-1.593912,-0.236255,-0.002224,-0.392776,-2.005520,-0.306309,-0.253465,1.539975,-0.407657,0.687552,-0.258684,1.995801,-1.554302,-0.531345,0.346683,0.104860,8.99,0
5,1.863194,3.853830,0.220877,-0.296455,0.207090,-0.487512,1.454433,-0.153871,-0.147691,0.606259,-0.311545,0.118870,0.611417,-0.387779,-0.003506,-0.061238,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85439,-0.753070,-1.448546,1.130943,1.167232,0.201294,-0.245013,0.113600,-1.027914,1.212622,-0.053734,0.050720,0.202703,0.725051,-1.016408,0.469657,0.304103,0.75,0
85440,-0.539994,0.514863,-0.043655,0.851781,-0.059080,0.319422,-0.573884,-1.541622,-0.192496,-0.702496,-0.173193,0.059047,0.441880,0.574042,-0.007875,-0.067811,40.00,0
85441,-1.833594,-1.464593,-0.448623,-0.496368,0.481275,-1.151529,-0.210526,0.895314,-0.274270,1.832300,0.754963,0.363656,0.279139,-0.118669,0.209083,-0.049870,140.00,0
